# Imports

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Setup
sns.set_theme(style="whitegrid")

# Extração dos dados do PostgreSQL

In [ ]:
# --- CONFIGURAÇÃO DE CONEXÃO COM POSTGRESQL ---
# As variáveis são lidas do ambiente Docker injetado pelo docker-compose
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_DB = os.environ.get("POSTGRES_DB")
DB_HOST = os.environ.get("DB_HOST") 
RAW_TABLE_NAME = "heart_disease_raw" # A tabela que o FastAPI carrega

DATABASE_URL = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{DB_HOST}:5432/{POSTGRES_DB}"
engine = create_engine(DATABASE_URL)

# --- AÇÃO CHAVE: LER A TABELA PERSISTIDA PARA O DATAFRAME 'df' ---
try:
    # Lendo o DataFrame diretamente da tabela que contém os dados brutos
    df = pd.read_sql(text(f"SELECT * FROM {RAW_TABLE_NAME}"), engine)
    
    print(f"VERIFICAÇÃO: DataFrame 'df' lido com sucesso da tabela PostgreSQL: {RAW_TABLE_NAME}")
    print(f"Total de linhas para limpeza: {len(df)}")

except Exception as e:
    print("\n" + "="*50)
    print("ERRO CRÍTICO AO LER DO POSTGRESQL!")
    print("O FastAPI /load_raw_to_db/ falhou ou o pg_db não está pronto.")
    print(f"Erro: {e}")
    print("="*50)
    exit()

# Tratamento dos Dados

## Limpeza

In [ ]:
# ===== 1. Remoção das duplicatas =====
df_sem_duplicatas = df.drop_duplicates().reset_index(drop=True)

print("Linhas antes:", len(df))
print("Linhas depois da remoção de duplicatas:", len(df_sem_duplicatas))

# ===== 2. Substituição dos valores impossíveis =====
# Valores impossíveis identificados:
# resting bp s: 0
# cholesterol: 0

df_clean = df_sem_duplicatas.copy()

# Substituir resting bp s = 0 pela mediana (ignorando os zeros)
rbp_median = df_clean.loc[df_clean["resting bp s"] > 0, "resting bp s"].median()
df_clean.loc[df_clean["resting bp s"] == 0, "resting bp s"] = rbp_median

# Substituir cholesterol = 0 pela mediana
chol_median = df_clean.loc[df_clean["cholesterol"] > 0, "cholesterol"].median()
df_clean.loc[df_clean["cholesterol"] == 0, "cholesterol"] = chol_median

## Encoding

In [ ]:
# Colunas categóricas a converter
categorical_cols = ["chest pain type", "resting ecg", "ST slope"]

# OneHotEncoder com drop='first' evita multicolinearidade
ohe = OneHotEncoder(drop=None, sparse_output=False)

# Ajustar e transformar
encoded = ohe.fit_transform(df_clean[categorical_cols])

# Novo dataframe codificado
encoded_df = pd.DataFrame(encoded, 
                          columns=ohe.get_feature_names_out(categorical_cols))

# Remover colunas originais e substituir pelas novas
df_encoded = pd.concat([df_clean.drop(columns=categorical_cols).reset_index(drop=True),
                        encoded_df.reset_index(drop=True)], axis=1)

## Scaling

In [ ]:
# Separar numericas e categoricas
num_cols = ["age", "resting bp s", "cholesterol",
            "fasting blood sugar", "max heart rate", "oldpeak"]

# Copiar dataset para não sujar o original
df_scaled = df_encoded.copy()

# Aplicar o scaler
scaler = StandardScaler()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])

# Split dos Dados

In [ ]:
df_final = df_scaled.copy()